In [1]:
import xarray as xr
import json

In [2]:
# load dataset
data = xr.open_dataset(r"C:\Users\rauc8872\Documents\carbon_dioxide\CO2_jan_05101520.nc")
data

<xarray.Dataset>
Dimensions:    (longitude: 480, latitude: 241, level: 18, time: 4)
Coordinates:
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 ... 357.0 357.8 358.5 359.2
  * latitude   (latitude) float32 90.0 89.25 88.5 87.75 ... -88.5 -89.25 -90.0
  * level      (level) int32 30 50 70 100 150 200 ... 800 850 900 925 950 1000
  * time       (time) datetime64[ns] 2005-01-01 2010-01-01 2015-01-01 2020-01-01
Data variables:
    co2        (time, level, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-04-12 10:19:52 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [3]:
# convert mass mixing ratio to volume mixing ratio according to
# https://confluence.ecmwf.int/pages/viewpage.action?pageId=153391710

vmmr_data = data.assign(co2 = (data["co2"] * 28.9644 / 44.0095 * 1e+6))
vmmr_data["co2"].attrs["units"] = "ppmv"
vmmr_data["co2"].attrs["long_name"] = "Carbon dioxide volume mixing ration"
vmmr_data["co2"].attrs["standard_name"] = "volume_fraction_of_carbon_dioxide_in_air"
vmmr_data

<xarray.Dataset>
Dimensions:    (longitude: 480, latitude: 241, level: 18, time: 4)
Coordinates:
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 ... 357.0 357.8 358.5 359.2
  * latitude   (latitude) float32 90.0 89.25 88.5 87.75 ... -88.5 -89.25 -90.0
  * level      (level) int32 30 50 70 100 150 200 ... 800 850 900 925 950 1000
  * time       (time) datetime64[ns] 2005-01-01 2010-01-01 2015-01-01 2020-01-01
Data variables:
    co2        (time, level, latitude, longitude) float32 367.5 367.5 ... 408.1
Attributes:
    Conventions:  CF-1.6
    history:      2023-04-12 10:19:52 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [4]:
# The dataset has 25 pressure levels which correspond to the altitude values below
# https://confluence.ecmwf.int/display/UDOC/L60+model+level+definitions
# We want to assign the altitude values instead of the pressure levels so we can regularize the altitude afterwards.
height_values = [23000, 20000, 18200, 16000, 13500, 11800, 10400, 8900, 7400, 
5700, 4500, 3200, 2000, 1400, 1133, 885, 546, 10]

vmmr_data = vmmr_data.assign_coords(level = height_values)
vmmr_data["level"].attrs["units"] = "m"
vmmr_data["level"].attrs["long_name"] = "height_values"
vmmr_data

<xarray.Dataset>
Dimensions:    (longitude: 480, latitude: 241, level: 18, time: 4)
Coordinates:
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 ... 357.0 357.8 358.5 359.2
  * latitude   (latitude) float32 90.0 89.25 88.5 87.75 ... -88.5 -89.25 -90.0
  * level      (level) int32 23000 20000 18200 16000 13500 ... 1133 885 546 10
  * time       (time) datetime64[ns] 2005-01-01 2010-01-01 2015-01-01 2020-01-01
Data variables:
    co2        (time, level, latitude, longitude) float32 367.5 367.5 ... 408.1
Attributes:
    Conventions:  CF-1.6
    history:      2023-04-12 10:19:52 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [5]:
vmmr_data_inverted = vmmr_data.reindex(level = list(reversed(vmmr_data.level)))
vmmr_data_inverted

<xarray.Dataset>
Dimensions:    (level: 18, longitude: 480, latitude: 241, time: 4)
Coordinates:
  * level      (level) int32 10 546 885 1133 1400 ... 16000 18200 20000 23000
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 ... 357.0 357.8 358.5 359.2
  * latitude   (latitude) float32 90.0 89.25 88.5 87.75 ... -88.5 -89.25 -90.0
  * time       (time) datetime64[ns] 2005-01-01 2010-01-01 2015-01-01 2020-01-01
Data variables:
    co2        (time, level, latitude, longitude) float32 387.2 387.2 ... 402.0
Attributes:
    Conventions:  CF-1.6
    history:      2023-04-12 10:19:52 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [17]:
# export to netcdf

vmmr_data_inverted.to_netcdf(r"C:\Users\rauc8872\Documents\carbon_dioxide\CO2_jan_05101520_inverted.nc", 
                encoding = {"longitude": {"_FillValue": 0}, "latitude": {"_FillValue": 0}, "co2": {"_FillValue": 0}})

In [2]:
#load regularized data
data = xr.open_dataset(r"C:\Users\rauc8872\Documents\carbon_dioxide\CO2_jan_05101520_inverted_regularized.nc")
data

<xarray.Dataset>
Dimensions:    (longitude: 480, latitude: 241, level: 41, time: 4)
Coordinates:
  * longitude  (longitude) float32 nan 0.75 1.5 2.25 ... 357.0 357.8 358.5 359.2
  * latitude   (latitude) float32 90.0 89.25 88.5 87.75 ... -88.5 -89.25 -90.0
  * level      (level) int32 10 510 1010 1510 2010 ... 18510 19010 19510 20010
  * time       (time) datetime64[ns] 2005-01-01 2010-01-01 2015-01-01 2020-01-01
Data variables:
    co2        (time, level, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-04-12 10:19:52 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [39]:
data.min()["co2"].item()

361.48724365234375

In [38]:
data.max()["co2"].item()

605.1389770507812

In [30]:
xr.plot.hist(data["co2"][:, :, :, :])

(array([5.4588650e+06, 1.1537653e+07, 1.9654920e+06, 7.6000000e+03,
       1.5740000e+03, 2.8500000e+02, 4.1000000e+01, 5.0000000e+00,
       3.0000000e+00, 2.0000000e+00]), array([361.48724, 385.85242, 410.2176 , 434.58276, 458.94794, 483.3131 ,
       507.67828, 532.04346, 556.4086 , 580.7738 , 605.139  ],
      dtype=float32), <BarContainer object of 10 artists>)

In [4]:
data_range = (data.min()["co2"].item(), data.max()["co2"].item())

In [3]:
# histograms for 2005, 2010, 2015, 2020
# xr.plot.hist(data["co2"][0, :, :, :], xlim=[360, 600])


statistics = []
for index in range(0, 4):
    histogram = data["co2"][index, :, :, :].plot.hist(bins=25, range = data_range)
    year = 2005 + index * 5
    statistics_year = dict()
    statistics_year["year"] = year
    statistics_year["mean"] = data.mean(dim=["level", "latitude", "longitude"])["co2"].item(0)
    statistics_year["std"] = data.std(dim=["level", "latitude", "longitude"])["co2"].item(0)
    statistics_year["counts"] = histogram[0].tolist()
    statistics_year["values"] = []
    for element in histogram[1].tolist():
        statistics_year["values"].append(round(element, 3))
    # statistics_year["values"] = list(map(lambda x: round(x), histogram[1].tolist()))
    statistics.append(statistics_year)
statistics

[{'year': 2005, 'mean': 378.24468994140625, 'std': 3.489166021347046, 'counts': [51783.0, 3875280.0, 795155.0, 15222.0, 2503.0, 1190.0, 847.0, 563.0, 192.0, 78.0, 42.0, 15.0, 8.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'values': [361.487, 371.233, 380.979, 390.725, 400.472, 410.218, 419.964, 429.71, 439.456, 449.202, 458.948, 468.694, 478.44, 488.186, 497.932, 507.678, 517.424, 527.17, 536.917, 546.663, 556.409, 566.155, 575.901, 585.647, 595.393, 605.139]}, {'year': 2010, 'mean': 378.24468994140625, 'std': 3.489166021347046, 'counts': [0.0, 145.0, 4054426.0, 669684.0, 12748.0, 2874.0, 1328.0, 765.0, 476.0, 198.0, 114.0, 55.0, 38.0, 20.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'values': [361.487, 371.233, 380.979, 390.725, 400.472, 410.218, 419.964, 429.71, 439.456, 449.202, 458.948, 468.694, 478.44, 488.186, 497.932, 507.678, 517.424, 527.17, 536.917, 546.663, 556.409, 566.155, 575.901, 585.647, 595.393, 605.139]}, {'year': 2015, 'mean': 378.2446

In [6]:
histogram = data["co2"][index, :, :, :].plot.hist(bins=25, range=data_range)
statistics_total = dict()
statistics_total["mean"] = data.mean()["co2"].item(0)
statistics_total["std"] = data.std()["co2"].item(0)
statistics_total["counts"] = histogram[0].tolist()
statistics_total["values"] = []
for element in histogram[1].tolist():
    statistics_total["values"].append(round(element, 3))
statistics.append(statistics_total)
statistics

[{'year': 2005, 'mean': 378.24468994140625, 'std': 3.489166021347046, 'counts': [51783.0, 3875280.0, 795155.0, 15222.0, 2503.0, 1190.0, 847.0, 563.0, 192.0, 78.0, 42.0, 15.0, 8.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'values': [361.487, 371.233, 380.979, 390.725, 400.472, 410.218, 419.964, 429.71, 439.456, 449.202, 458.948, 468.694, 478.44, 488.186, 497.932, 507.678, 517.424, 527.17, 536.917, 546.663, 556.409, 566.155, 575.901, 585.647, 595.393, 605.139]}, {'year': 2010, 'mean': 378.24468994140625, 'std': 3.489166021347046, 'counts': [0.0, 145.0, 4054426.0, 669684.0, 12748.0, 2874.0, 1328.0, 765.0, 476.0, 198.0, 114.0, 55.0, 38.0, 20.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'values': [361.487, 371.233, 380.979, 390.725, 400.472, 410.218, 419.964, 429.71, 439.456, 449.202, 458.948, 468.694, 478.44, 488.186, 497.932, 507.678, 517.424, 527.17, 536.917, 546.663, 556.409, 566.155, 575.901, 585.647, 595.393, 605.139]}, {'year': 2015, 'mean': 378.2446

In [90]:
with open(r"C:\Users\rauc8872\Documents\carbon_dioxide\statistics.json", "w") as outfile:
    outfile.write(json.dumps(statistics))

In [66]:
# histogram for 2020
data["co2"][0, :, :, :].plot.hist(bins=10)

(array([4.771160e+05, 4.212315e+06, 4.778200e+04, 3.140000e+03,
       1.317000e+03, 8.410000e+02, 2.410000e+02, 8.400000e+01,
       3.300000e+01, 1.100000e+01]), array([361.48724, 374.31433, 387.1414 , 399.96844, 412.79553, 425.62262,
       438.44968, 451.27673, 464.10382, 476.9309 , 489.75797],
      dtype=float32), <BarContainer object of 10 artists>)

In [34]:
data.std(dim=["level", "latitude", "longitude"])["co2"].item(0)

3.489166021347046

In [16]:
data.median(dim=["level", "latitude", "longitude"])

<xarray.Dataset>
Dimensions:  (time: 4)
Coordinates:
  * time     (time) datetime64[ns] 2005-01-01 2010-01-01 2015-01-01 2020-01-01
Data variables:
    co2      (time) float32 377.9 387.5 396.4 409.7

In [24]:
data.quantile(q=0.1, dim=["level", "latitude", "longitude"])

<xarray.Dataset>
Dimensions:   (time: 4)
Coordinates:
  * time      (time) datetime64[ns] 2005-01-01 2010-01-01 2015-01-01 2020-01-01
    quantile  float64 0.1
Data variables:
    co2       (time) float64 374.3 384.9 393.5 405.3

In [26]:
data.quantile(q=0.99, dim=["level", "latitude", "longitude"])

<xarray.Dataset>
Dimensions:   (time: 4)
Coordinates:
  * time      (time) datetime64[ns] 2005-01-01 2010-01-01 2015-01-01 2020-01-01
    quantile  float64 0.99
Data variables:
    co2       (time) float64 387.4 396.0 406.4 420.5